# Alternative Tracing Methods

![AWTT](../../images/alternative_ways_to_trace_0.png)

So far in this module, we've taken a look at the traceable decorator, and how we can use it to set up tracing.

In this lesson, we're going to look at alternative ways in which we can set up tracing, and when you should think about using these different approaches.

## LangChain and LangGraph

If we are using LangChain or LangGraph, all we need to do to set up tracing is to set a few environment variables

![AWTT](../../images/alternative_ways_to_trace_1.png)

In [ ]:
# You can set them inline
import os
os.environ["OPENAI_API_KEY"] = ""
os.environ["LANGSMITH_API_KEY"] = ""
os.environ["LANGSMITH_TRACING"] = "true"
os.environ["LANGSMITH_PROJECT"] = "langsmith-academy"  # If you don't set this, traces will go to the Default project

In [ ]:
# Or you can use a .env file
from dotenv import load_dotenv
load_dotenv(dotenv_path="../../.env", override=True)

False

Don't worry too much about our graph implementation here, you can learn more about LangGraph through our LangGraph Academy course!

In [ ]:
!pip install --upgrade langchain openai langchain_community nest_asyncio


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 964.9/964.9 kB 10.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 65.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.7/64.7 kB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 4.9 MB/s eta 0:00:00
  Attempting uninstall: requests
    Found existing installation: requests 2.32.4
    Uninstalling requests-2.32.4:
      Successfully uninstalled requests-2.32.4
  Attempting uninstall: openai
    Found existing installation: openai 1.109.1
    Uninstalling openai-1.109.1:
      Successfully uninstalled openai-1.109.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires requests==2.32.4, but you have requests 2.32.5 which is incompatible.


In [8]:
import os
os.environ["OPENAI_API_KEY"] = "sk-your-api-key-here"  # <-- Add your API key here

import nest_asyncio
import operator
from langchain.schema import Document, HumanMessage, BaseMessage
from langchain.chat_models import ChatOpenAI
from IPython.display import display
from typing import List
from typing_extensions import TypedDict, Annotated
import networkx as nx
import matplotlib.pyplot as plt

# Dummy retriever
class DummyDoc:
    def __init__(self, page_content: str):
        self.page_content = page_content

class DummyRetriever:
    def __init__(self):
        self.documents = [
            "LangSmith helps trace and debug LLM applications.",
            "OpenAI models like GPT-4o-mini can generate text and perform reasoning.",
            "Vector databases are used in Retrieval-Augmented Generation (RAG) systems.",
            "The @traceable decorator enables LangSmith function tracing.",
        ]

    def invoke(self, query: str) -> List[DummyDoc]:
        matched = [
            DummyDoc(d) for d in self.documents
            if any(word.lower() in d.lower() for word in query.split())
        ]
        return matched or [DummyDoc(d) for d in self.documents]

def get_vector_db_retriever() -> DummyRetriever:
    return DummyRetriever()

RAG_PROMPT = """Context:\n{context}\n\nConversation:\n{conversation}\n\nQuestion: {question}"""

# Initialization
nest_asyncio.apply()
retriever = get_vector_db_retriever()
llm = ChatOpenAI(model_name="gpt-4o-mini", temperature=0)

# Helper
def get_buffer_string(messages: List[BaseMessage]) -> str:
    return "\n".join([f"{m.__class__.__name__}: {m.content}" for m in messages])

# Graph state
class GraphState(TypedDict):
    question: str
    messages: Annotated[List[BaseMessage], operator.add]
    documents: List[Document]

# Nodes
def retrieve_documents(state: GraphState):
    messages = state.get("messages", [])
    question = state["question"]
    documents = retriever.invoke(f"{get_buffer_string(messages)} {question}")
    return {"documents": documents}

def generate_response(state: GraphState):
    question = state["question"]
    messages = state.get("messages", [])
    documents = state["documents"]
    formatted_docs = "\n\n".join(doc.page_content for doc in documents)

    rag_prompt_formatted = RAG_PROMPT.format(context=formatted_docs, conversation=messages, question=question)
    generation = llm([HumanMessage(content=rag_prompt_formatted)])

    return {"documents": documents, "messages": [HumanMessage(question), generation]}

# Execute workflow
state = {"question": "Explain the @traceable decorator", "messages": [], "documents": []}
state.update(retrieve_documents(state))
state.update(generate_response(state))

# Print response
print("AI Response:")
print(state["messages"][-1].content)

# Visualize workflow
G = nx.DiGraph()
G.add_edge("START", "retrieve_documents")
G.add_edge("retrieve_documents", "generate_response")
G.add_edge("generate_response", "END")

plt.figure(figsize=(6, 4))
nx.draw(G, with_labels=True, node_size=3000, node_color="lightblue", arrows=True)
plt.title("RAG Workflow Graph")
plt.show()


SyntaxError: invalid syntax (ipython-input-1876171567.py, line 3)

We're setting up a simple graph in LangGraph. If you want to learn more about LangGraph, I would highly recommend taking a look at our LangGraph Academy course.

You can also pass in metadata or other fields through an optional config

In [9]:
question = "How do I set up tracing if I'm using LangChain?"
simple_rag_graph.invoke({"question": question}, config={"metadata": {"foo": "bar"}})

NameError: name 'simple_rag_graph' is not defined

##### Let's take a look in LangSmith!

## Tracing Context Manager

In Python, you can use the trace context manager to log traces to LangSmith. This is useful in situations where:

You want to log traces for a specific block of code.
You want control over the inputs, outputs, and other attributes of the trace.
It is not feasible to use a decorator or wrapper.
Any or all of the above.
The context manager integrates seamlessly with the traceable decorator and wrap_openai wrapper, so you can use them together in the same application.

You still need to set your `LANGSMITH_API_KEY` and `LANGSMITH_TRACING`

![AWTT](../../images/alternative_ways_to_trace_2.png)

In [24]:
import os
from dotenv import load_dotenv
from langsmith import trace, traceable
from openai import OpenAI
from typing import List
import nest_asyncio
from utils import get_vector_db_retriever

# Load environment variables from your .env file
load_dotenv(dotenv_path="../../.env", override=True)

# Set tracing environment variable
os.environ["LANGSMITH_TRACING"] = "false"  # using manual RunTree control
# Ensure API keys are set in your environment or in your .env file

# Initialize OpenAI client
openai_client = OpenAI()
nest_asyncio.apply()
retriever = get_vector_db_retriever()


In [25]:
@traceable
def retrieve_documents(question: str):
    documents = retriever.invoke(question)
    return documents

@traceable
def generate_response(question: str, documents):
    formatted_docs = "\n\n".join(doc.page_content for doc in documents)
    messages = [
        {
            "role": "system",
            "content": RAG_SYSTEM_PROMPT
        },
        {
            "role": "user",
            "content": f"Context: {formatted_docs} \n\n Question: {question}"
        }
    ]
    response = call_openai(messages)
    return response

@traceable
def call_openai(messages: List[dict], model: str = MODEL_NAME, temperature: float = 0.0):
    response = openai_client.chat.completions.create(
        model=model,
        messages=messages,
        temperature=temperature,
    )
    return response


NameError: name 'MODEL_NAME' is not defined

In [26]:
@traceable
def langsmith_rag(question: str):
    documents = retrieve_documents(question)
    response = generate_response(question, documents)
    return response.choices[0].message.content

question = "How do I trace with tracing context?"
ai_answer = langsmith_rag(question)
print(ai_answer)


NameError: name 'RAG_SYSTEM_PROMPT' is not defined

## wrap_openai

The wrap_openai/wrapOpenAI methods in Python/TypeScript allow you to wrap your OpenAI client in order to automatically log traces -- no decorator or function wrapping required! The wrapper works seamlessly with the @traceable decorator or traceable function and you can use both in the same application.

You still need to set your `LANGSMITH_API_KEY` and `LANGSMITH_TRACING`

![AWTT](../../images/alternative_ways_to_trace_3.png)

In [21]:
# Imports and Constants

# TODO: Import wrap_openai from langsmith.wrappers when ready
# from langsmith.wrappers import wrap_openai
import openai
from typing import List
import nest_asyncio
from utils import get_vector_db_retriever

MODEL_PROVIDER = "openai"
MODEL_NAME = "gpt-4o-mini"
APP_VERSION = 1.0

RAG_SYSTEM_PROMPT = """You are an assistant for question-answering tasks.
Use the following pieces of retrieved context to answer the latest question in the conversation.
If you don't know the answer, just say that you don't know.
Use three sentences maximum and keep the answer concise.
"""


ModuleNotFoundError: No module named 'utils'

In [22]:
#  Client Initialization and Setup

# TODO: Wrap the OpenAI Client once ready
openai_client = openai.Client()

nest_asyncio.apply()
retriever = get_vector_db_retriever()


The wrapped OpenAI client accepts all the same langsmith_extra parameters as @traceable decorated functions

In [23]:
#  Document Retriever Function

from langsmith import traceable

@traceable(run_type="chain")
def retrieve_documents(question: str):
    return retriever.invoke(question)


## [Advanced] RunTree

Another, more explicit way to log traces to LangSmith is via the RunTree API. This API allows you more control over your tracing - you can manually create runs and children runs to assemble your trace. You still need to set your `LANGSMITH_API_KEY`, but `LANGSMITH_TRACING` is not necessary for this method.

![AWTT](../../images/alternative_ways_to_trace_4.png)

In [20]:
# Environment Setup and Imports

import os
from dotenv import load_dotenv
from langsmith import RunTree, utils
from openai import OpenAI
from typing import List
import nest_asyncio
from utils import get_vector_db_retriever

# Load environment variables from .env file if present
load_dotenv(dotenv_path="../../.env", override=True)

os.environ["LANGSMITH_TRACING"] = "false"  # Disable auto-tracing, we use RunTree explicitly

# Sanity check tracing status
print("Tracing enabled?", utils.tracing_is_enabled())  # Should print False

openai_client = OpenAI()
nest_asyncio.apply()
retriever = get_vector_db_retriever()


ModuleNotFoundError: No module named 'utils'

In [ ]:
# Document Retrieval Function

def retrieve_documents(parent_run: RunTree, question: str):
    # Create a child run with added metadata
    child_run = parent_run.create_child(
        name="Retrieve Documents",
        run_type="retriever",
        inputs={"question": question},
        metadata={"version": "1.0", "component": "retriever"}
    )
    documents = retriever.invoke(question)
    print(f"Documents retrieved: {[doc.page_content for doc in documents]}")  # Debug log
    child_run.end(outputs={"documents": documents})
    child_run.post()
    return documents


Let's go ahead and set `LANGSMITH_TRACING` to false, as we are using RunTree to manually create runs in this case.

In [19]:
# Response Generation Function with Document Formatting as Child Run

def generate_response(parent_run: RunTree, question: str, documents, model: str = "gpt-4o-mini", temperature: float = 0.0):
    # Split document formatting into own sub-run for extra granularity
    prep_run = parent_run.create_child(
        name="Format Documents",
        run_type="parser",
        inputs={"documents": documents},
        metadata={"version": "1.0", "component": "formatter"}
    )
    formatted_docs = "\n\n".join(doc.page_content for doc in documents)
    prep_run.end(outputs={"formatted_docs": formatted_docs})
    prep_run.post()

    rag_system_prompt = """You are an assistant for question-answering tasks.
    Use the following pieces of retrieved context to answer the latest question in the conversation.
    If you don't know the answer, just say that you don't know.
    Use three sentences maximum and keep the answer concise.
    """
    # Create a child run for the chain step with extra metadata and inputs
    child_run = parent_run.create_child(
        name="Generate Response",
        run_type="chain",
        inputs={"question": question, "formatted_docs": formatted_docs, "model": model, "temperature": temperature},
        metadata={"version": "1.0", "component": "chain"}
    )
    messages = [
        {
            "role": "system",
            "content": rag_system_prompt
        },
        {
            "role": "user",
            "content": f"Context: {formatted_docs} \n\n Question: {question}"
        }
    ]
    try:
        openai_response = call_openai(child_run, messages, model=model, temperature=temperature)
        child_run.end(outputs={"openai_response": openai_response})
    except Exception as e:
        child_run.end(outputs={"error": str(e)}, status="error")
        raise
    child_run.post()
    return openai_response


We have rewritten our RAG application, except this time we pass a RunTree argument through our function calls, and create child runs at each layer. This gives our RunTree the same hierarchy that we were automatically able to establish with @traceable

In [18]:
# OpenAI Call Function with Timing and Error Handling

def call_openai(
    parent_run: RunTree, messages: List[dict], model: str = "gpt-4o-mini", temperature: float = 0.0
) -> str:
    # Create a child run for the OpenAI call with metadata and timing
    child_run = parent_run.create_child(
        name="OpenAI Call",
        run_type="llm",
        inputs={"messages": messages, "model": model, "temperature": temperature},
        metadata={"version": "1.0", "component": "llm"}
    )
    import time
    start_time = time.time()

    openai_response = openai_client.chat.completions.create(
        model=model,
        messages=messages,
        temperature=temperature,
    )

    elapsed_time = time.time() - start_time
    child_run.end(outputs={"openai_response": openai_response, "elapsed_time_secs": elapsed_time})
    child_run.post()
    return openai_response


In [17]:
# Root RAG Function with RunTree Management

def langsmith_rag(question: str, model: str = "gpt-4o-mini", temperature: float = 0.0):
    # Root RunTree with extra metadata and inputs, supports passing model params
    root_run_tree = RunTree(
        name="Chat Pipeline",
        run_type="chain",
        inputs={"question": question, "model": model, "temperature": temperature},
        metadata={"version": "1.0", "app": "langsmith_rag"}
    )

    # Pass RunTree and params into calls
    documents = retrieve_documents(root_run_tree, question)
    response = generate_response(root_run_tree, question, documents, model=model, temperature=temperature)
    output = response.choices[0].message.content

    root_run_tree.end(outputs={"generation": output})
    root_run_tree.post()
    return output
